# SI 618 Fall 2022 Homework 3 (100 points)

## Part 1 (80 points) 

The provided ‘movie_actors_data.txt’ file contains a JSON string on each line. For example, the first line is: 

>{"rating": 9.3, "genres": ["Crime", "Drama"], "rated": "R", "filming_locations": "Ashland, Ohio, USA", "language": ["English"], "title": "The Shawshank Redemption", "runtime": ["142 min"], "poster": "http://img3.douban.com/lpic/s1311361.jpg", "imdb_url": "http://www.imdb.com/title/tt0111161/", "writers": ["Stephen King", "Frank Darabont"], "imdb_id": "tt0111161", "directors": ["Frank Darabont"], "rating_count": 894012, "actors": ["Tim Robbins", "Morgan Freeman", "Bob Gunton", "William Sadler", "Clancy Brown", "Gil Bellows", "Mark Rolston", "James Whitmore", "Jeffrey DeMunn", "Larry Brandenburg", "Neil Giuntoli", "Brian Libby", "David Proval", "Joseph Ragno", "Jude Ciccolella"], "plot_simple": "Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.", "year": 1994, "country": ["USA"], "type": "M", "release_date": 19941014, "also_known_as": ["Die Verurteilten"]}

The fields we are interested in are imdb_id, title , rating, genres, actors, directors, year, and country. You will parse the JSON strings, and load the data into three tables in SQLite, and then write SQL queries to retrieve the data specified. 

You will create 4 tables: 
* The “movies” table, which has four columns: imdb_id, title, year, rating, country  

* The “movie_genre” table, which has two columns: imdb_id and genre. A movie typically has multiple genres, and in this case, there should be one row for each genre. If some movie does not have any genre, ignore that movie.  

* The “movie_actor” table, which has two columns imdb_id and actor. A movie typically has multiple actors, and in this case, there should be one row for each actor. 

* The “movie_director” table, which has two columns imdb_id and director. A movie typically has multiple directors, and in this case, there should be one row for each director. 

**When you run your Python code, it should print out EXACTLY the output provided.**

#### Q1. (10 points) Parse input file to get needed data for the three tables and load them into appropriate Python data structure. 

In [1]:
from bs4 import BeautifulSoup
import json
import urllib3
import pandas as pd

In [2]:
json_movie = open("movie_actors_data.txt",'r')
all_movie = []
for s in json_movie.readlines():
    all_movie.append(json.loads(s))

#### Q2. (5 points) Create the movies table and load data into it. Note that you only need to consider the first country if there are multiple countries associated with a movie. Read and show the first 5 rows of the movies table in your SQLite database.

In [3]:
movie_lst = []
for jsondata in all_movie:
    dataset = [jsondata['imdb_id'],jsondata['title'],jsondata['year'],jsondata['rating'],jsondata['country'][0]]
    movie_lst.append(dataset)
movies = pd.DataFrame(movie_lst, columns=['imdb_id','title','year','rating','country'])

In [4]:
movies.head(5)

,imdb_id,title,year,rating,country
0,tt0111161,The Shawshank Redemption,1994,9.3,USA
1,tt0068646,The Godfather,1972,9.2,USA
2,tt0071562,The Godfather: Part II,1974,9.1,USA
3,tt0110912,Pulp Fiction,1994,9.0,USA
4,tt0060196,"Il buono, il brutto, il cattivo.",1966,9.0,Italy


#### Q3. (5 points) Create the movie_genre table and load data into it.

In [5]:
genre_lst = []
for jsondata in all_movie:
    if jsondata['genres']:
        for genre in jsondata['genres']:
            genre = [jsondata['imdb_id'],genre]
            genre_lst.append(genre)

movie_genre = pd.DataFrame(genre_lst, columns=['imdb_id','genre'])

In [6]:
movie_genre.head(20)

,imdb_id,genre
0,tt0111161,Crime
1,tt0111161,Drama
2,tt0068646,Crime
3,tt0068646,Drama
4,tt0071562,Crime
5,tt0071562,Drama
6,tt0110912,Crime
7,tt0110912,Thriller
8,tt0060196,Adventure
9,tt0060196,Western


#### Q4. (5 points) Create the movie_actor table and load data into it.

In [7]:
actor_lst = []
for jsondata in all_movie:
    for actor in jsondata['actors']:
        row = [jsondata['imdb_id'], actor]
        actor_lst.append(row)
        
movie_actor = pd.DataFrame(actor_lst, columns = ['imdb_id', 'actor'])

In [8]:
movie_actor.head(10)

,imdb_id,actor
0,tt0111161,Tim Robbins
1,tt0111161,Morgan Freeman
2,tt0111161,Bob Gunton
3,tt0111161,William Sadler
4,tt0111161,Clancy Brown
5,tt0111161,Gil Bellows
6,tt0111161,Mark Rolston
7,tt0111161,James Whitmore
8,tt0111161,Jeffrey DeMunn
9,tt0111161,Larry Brandenburg


#### Q5. (5 points) Create the movie_director table and load data into it.

In [9]:
director_lst = []
for jsondata in all_movie:
    for director in jsondata['directors']:
        row = [jsondata['imdb_id'], director]
        director_lst.append(row)
        
movie_director = pd.DataFrame(director_lst, columns = ['imdb_id', 'director'])

In [10]:
movie_director.head(10)

,imdb_id,director
0,tt0111161,Frank Darabont
1,tt0068646,Francis Ford Coppola
2,tt0071562,Francis Ford Coppola
3,tt0110912,Quentin Tarantino
4,tt0060196,Sergio Leone
5,tt0050083,Sidney Lumet
6,tt0468569,Christopher Nolan
7,tt0108052,Steven Spielberg
8,tt0167260,Peter Jackson
9,tt0137523,David Fincher


#### Q6. (10 points) Write a SQL query to find the number of released movies by decade in chronological order. (Note: you will need to round *down* years to get the corresponding decade. e.g. 2009 belongs to the 2000s)
```
Number of movies broken down by decade: Decade, Number
1920, 5
1930, 8
1940, 16
1950, 30
1960, 23
1970, 24
1980, 26
1990, 38
2000, 57
2010, 21

```

In [11]:
import sqlite3
import math

In [12]:
movie_conn = sqlite3.connect('moviessql')
movie_c = movie_conn.cursor()
movie_c.execute('CREATE TABLE IF NOT EXISTS movies (imdb_id, title, year, rating, country)')
movie_conn.commit()

In [13]:
movies.to_sql('movies', movie_conn, if_exists='replace', index = False)

In [14]:
movie_c.execute('''
SELECT CAST((year/10)*10 as int),COUNT(imdb_id) FROM movies GROUP BY CAST((year/10)*10 as int)
''')

In [15]:
for row in movie_c.fetchall():
    print(row)

(1920, 5)
(1930, 8)
(1940, 16)
(1950, 30)
(1960, 23)
(1970, 24)
(1980, 26)
(1990, 38)
(2000, 57)
(2010, 21)


#### Q7. (10 points) Write a SQL query to find the top 5 genres in the US by the average ratings of movies in that genre. Print out the results.
```
Top 5 genres in the US: Genre, AvgRatings
Action, 8.461290322580647
War, 8.413333333333332
Crime, 8.408695652173916
Music, 8.4
Sci-Fi, 8.385000000000002
```

In [16]:
movie_c.execute('CREATE TABLE IF NOT EXISTS genres (imdb_genre_id, genre )')

In [17]:
movie_genre.to_sql('genres', movie_conn, if_exists='replace', index = False)

In [18]:
movie_c.execute("SELECT genres.genre, AVG(movies.rating) FROM movies \
                LEFT OUTER JOIN genres ON genres.imdb_id=movies.imdb_id \
                WHERE movies.country='USA' \
                GROUP BY genres.genre \
                ORDER BY AVG(movies.rating) DESC \
                LIMIT 5")


In [19]:
for row in movie_c.fetchall():
    print(row)

('Action', 8.461290322580647)
('War', 8.413333333333332)
('Crime', 8.408695652173916)
('Music', 8.4)
('Sci-Fi', 8.385000000000002)


#### Q8. (10 points) Write a SQL query to find the top 10 actors who have worked with the director Christopher Nolan in the most movies. The actors are ordered in decreasing number of movies they worked with the director. In case of ties, the rows are ordered by actors’ names in alphabetical order.

```
The actors who have worked with Christopher Nolan the most: Name, Times
Michael Caine, 5
Christian Bale, 4
Cillian Murphy, 3
Gary Oldman, 3
Morgan Freeman, 3
Joseph Gordon-Levitt, 2
Ken Watanabe, 2
Larry Holden, 2
Marion Cotillard, 2
Mark Boone Junior, 2
```

In [20]:
movie_c.execute('CREATE TABLE IF NOT EXISTS actors (imdb_id, actor)')
movie_c.execute('CREATE TABLE IF NOT EXISTS directors (imdb_id, director)')

In [21]:
movie_actor.to_sql('actors', movie_conn, if_exists='replace', index = False)
movie_director.to_sql('directors', movie_conn, if_exists='replace', index = False)

In [22]:
movie_c.execute("SELECT actors.actor, COUNT(actors.imdb_id) FROM actors \
                LEFT OUTER JOIN directors ON actors.imdb_id=directors.imdb_id \
                WHERE directors.director='Christopher Nolan' \
                GROUP BY actors.actor \
                ORDER BY COUNT(actors.imdb_id) DESC, actors.actor ASC \
                LIMIT 10")

In [23]:
for row in movie_c.fetchall():
    print(row)

('Michael Caine', 5)
('Christian Bale', 4)
('Cillian Murphy', 3)
('Gary Oldman', 3)
('Morgan Freeman', 3)
('Joseph Gordon-Levitt', 2)
('Ken Watanabe', 2)
('Larry Holden', 2)
('Marion Cotillard', 2)
('Mark Boone Junior', 2)


#### Q9. (10 points) Write a SQL query to find the top 10 directors based on average ratings of the movies they directed between the years year 1980 and 2010 (begin and end years are included). Only consider those who directed at least 2 movies in this time range. For each director, show their name, average rating of the movies, and the number of movies they directed. Sort the result by descending average movie rating, then by director name in alphabetical order.

```
Between 1980 and 2010, top 10 highest rated directors based on average movie rating with at least 2 credits:
Director, Movies
Frank Darabont, 8.9, 2
David Fincher, 8.8, 2
Peter Jackson, 8.8, 3
Christopher Nolan, 8.64, 5
Robert Zemeckis, 8.6, 2
Steven Spielberg, 8.5, 5
Quentin Tarantino, 8.475, 4
Stanley Kubrick, 8.45, 2
James Cameron, 8.4, 3
Ridley Scott, 8.4, 2
```


In [24]:
movie_c.execute("SELECT directors.director, AVG(movies.rating),COUNT(movies.imdb_id) FROM directors \
                LEFT OUTER JOIN movies ON movies.imdb_id=directors.imdb_id \
                WHERE movies.year BETWEEN 1980 AND 2010 \
                GROUP BY directors.director \
                HAVING COUNT(CASE WHEN movies.year BETWEEN 1980 AND 2010 THEN 1 ELSE NULL END)>2 \
                ORDER BY AVG(movies.rating) DESC, directors.director ASC \
                LIMIT 10")

In [25]:
movie_c.execute("SELECT directors.director, AVG(movies.rating),COUNT(movies.imdb_id) FROM directors \
                LEFT OUTER JOIN movies ON movies.imdb_id=directors.imdb_id \
                WHERE movies.year BETWEEN 1980 AND 2010 \
                GROUP BY directors.director \
                HAVING COUNT(movies.imdb_id)>=2 AND movies.year BETWEEN 1980 AND 2010 \
                ORDER BY AVG(movies.rating) DESC, directors.director ASC \
                LIMIT 10")

In [26]:
for row in movie_c.fetchall():
    print(row)

('Frank Darabont', 8.9, 2)
('David Fincher', 8.8, 2)
('Peter Jackson', 8.8, 3)
('Christopher Nolan', 8.64, 5)
('Robert Zemeckis', 8.6, 2)
('Steven Spielberg', 8.5, 5)
('Quentin Tarantino', 8.475, 4)
('Stanley Kubrick', 8.45, 2)
('James Cameron', 8.4, 3)
('Ridley Scott', 8.4, 2)


#### Q10. (10 points) Write a SQL query for finding pairs of actors who co-starred in at least 2 highly rated (rating > 9) movies together. 

The pairs of names must be unique. This means that ‘actor A, actor B’ and ‘actor B, actor A’ are the same pair, so only one of them should appear. For each pair of actors you print out, the two actors must be ordered alphabetically. The pairs are ordered in decreasing number of movies they co-stared in. In case of ties, the rows are ordered by actors’ names. 

>You will need to join the movie_actor table with itself to get this data (in addition to another required join). It is a bit tricky. If you cannot do it with SQL statement, you can also write some Python code that works on the Python data structure that you used to create the movie_actor table. That’ll mean much more lines of code, and if you do it that way, you’ll get 5 points instead of 10 points. You will only get 10 points if you solve it with pure SQL.

```
Pairs of actors who co-stared in 2 or more highly rated movies: Actor A, Actor B, Co-stared Movies
Al Pacino, Diane Keaton, 2
Al Pacino, John Cazale, 2
Al Pacino, Robert Duvall, 2
Al Pacino, Talia Shire, 2
Diane Keaton, John Cazale, 2
Diane Keaton, Robert Duvall, 2
Diane Keaton, Talia Shire, 2
John Cazale, Robert Duvall, 2
John Cazale, Talia Shire, 2
Robert Duvall, Talia Shire, 2
```

In [27]:
movie_c.execute("SELECT actor1.actor, actor2.actor,COUNT(movies.imdb_id) FROM actors actor1, actors actor2\
                LEFT OUTER JOIN movies ON actor1.imdb_id=movies.imdb_id\
                WHERE actor1.actor!=actor2.actor AND actor1.imdb_id=actor2.imdb_id \
                AND movies.rating>9 AND actor1.actor<actor2.actor\
                GROUP BY actor1.actor, actor2.actor \
                ORDER BY COUNT(movies.imdb_id) DESC\
                LIMIT 10")

In [28]:
for row in movie_c.fetchall():
    print(row)

('Al Pacino', 'Diane Keaton', 2)
('Al Pacino', 'John Cazale', 2)
('Al Pacino', 'Robert Duvall', 2)
('Al Pacino', 'Talia Shire', 2)
('Diane Keaton', 'John Cazale', 2)
('Diane Keaton', 'Robert Duvall', 2)
('Diane Keaton', 'Talia Shire', 2)
('John Cazale', 'Robert Duvall', 2)
('John Cazale', 'Talia Shire', 2)
('Robert Duvall', 'Talia Shire', 2)


## Part 2 (20 points) 

Write a python function that takes two arguments: genre and k 

The function should print out the top k directors who made the highest rated movies (on average) in the provided genre. You should use the sqlite3 database file you created in Part 1. Some example runs of my function are shown below. Your program should produce the same output when provided with the same command line arguments.

`get_top_genre_directors("Sci-Fi", 3)`
```
Top 3 directors who made highest rated Sci-Fi movies:
Director, Average Sci-Fi movie rating
Christopher Nolan, 8.8
George Lucas, 8.8
Irvin Kershner, 8.8
```

`get_top_genre_directors("War", 5)`
```
Top 5 directors who made highest rated War movies:
Director, Average War movie rating
Steven Spielberg, 8.75
Michael Curtiz, 8.7
Francis Ford Coppola, 8.6
Charles Chaplin, 8.5
Roberto Benigni, 8.5
```

`get_top_genre_directors("Animation", 10)`
```
Top 10 directors who made highest rated Animation movies:
Director, Average Animation movie rating
Isao Takahata, 8.4
Rob Minkoff, 8.4
Roger Allers, 8.4
Andrew Stanton, 8.3
Bob Peterson, 8.3
John Lasseter, 8.3
Lee Unkrich, 8.3
Adam Elliot, 8.2
Chris Sanders, 8.2
Dean DeBlois, 8.2
```

In [29]:
def get_top_genre_directors(genre, num):
    movie_c.execute("SELECT directors.director, AVG(movies.rating) FROM directors \
                    LEFT OUTER JOIN movies ON movies.imdb_id=directors.imdb_id \
                    LEFT OUTER JOIN genres ON genres.imdb_id=movies.imdb_id \
                    WHERE genres.genre=? \
                    GROUP BY directors.director \
                    ORDER BY AVG(movies.rating) DESC,directors.director ASC\
                    LIMIT ?",[(genre),(num),])
    for row in movie_c.fetchall():
        print(row)

In [30]:
get_top_genre_directors("War", 3)

('Steven Spielberg', 8.75)
('Michael Curtiz', 8.7)
('Francis Ford Coppola', 8.6)


In [31]:
get_top_genre_directors("Sci-Fi", 3)

('Christopher Nolan', 8.8)
('George Lucas', 8.8)
('Irvin Kershner', 8.8)


In [32]:
get_top_genre_directors("War", 5)

('Steven Spielberg', 8.75)
('Michael Curtiz', 8.7)
('Francis Ford Coppola', 8.6)
('Charles Chaplin', 8.5)
('Roberto Benigni', 8.5)


In [33]:
get_top_genre_directors("Animation", 10)

('Isao Takahata', 8.4)
('Rob Minkoff', 8.4)
('Roger Allers', 8.4)
('Andrew Stanton', 8.3)
('Bob Peterson', 8.3)
('John Lasseter', 8.3)
('Lee Unkrich', 8.3)
('Adam Elliot', 8.2)
('Chris Sanders', 8.2)
('Dean DeBlois', 8.2)


## What to submit: 
Please perform the following operations and turn in the corresponding files to canvas:
1. Jupyter notebook titled uniqname_si618_hw3.ipynb and the corresponding html page (uniqname_si618_hw3.html) 
through Canvas.
2. The sqlite database file you created. (uniqname_si618_hw3.db)